In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
import copy
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import sklearn.metrics as metrics

pd.set_option('display.max_rows', 100)

plt.rcParams['font.family'] = 'NanumBarunGothic'

#데이터를 불러온다.
df_all_real = pd.read_excel('/gdrive/MyDrive/data_science/train.xlsx')
df_all_test = pd.read_excel('/gdrive/MyDrive/data_science/test.xlsx')
 
df_all = copy.deepcopy(df_all_real)

In [ ]:
#데이터 전처리#
#필요없는 데이터 삭제

del df_all['검사결과코드']
# del df_all['핵심적발']
# del df_all['우범여부']
del df_all['신고일자']
del df_all['신고번호']
del df_all['수입자부호']
del df_all['신고인부호']
del df_all['과세가격원화금액']
del df_all['관세율']

In [ ]:
#데이터 전처리#
#결측값 제거
df_all.fillna('others',inplace=True)

#연속형 데이터 log scale로 변경
numeric_columns = ['신고중량(KG)']

for col in numeric_columns:
    df_all[col] = df_all[col].apply(lambda x: np.log1p(x))

#신고중량을 구간별로 나누어 준다.
group_list = list(range(0,11,1))
labeling = ['group'+str(x+1) for x in group_list]
df_all['신고중량(KG)'] = pd.cut(df_all['신고중량(KG)'],group_list,right=False,labels=labeling[:-1])
df_all['신고중량(KG)'] = df_all['신고중량(KG)'].astype('str')
df_all['신고중량(KG)'].fillna('others',inplace=True)

#HS10단위부호 집합의 크기를 줄여준다.
df_all['HS10단위부호'] = df_all['HS10단위부호'].astype('str')
df_all['HS10단위부호'] = df_all['HS10단위부호'].str[0:2]+df_all['HS10단위부호'].str[6:10]

In [ ]:
#데이터 전처리#
#feature 전부를 category 사용 위해 타입 변경

col_list = ['통관지세관부호', '해외거래처부호', '특송업체부호', '수입통관계획코드', '수입신고구분코드', '수입거래구분코드',
       '수입종류코드', '징수형태코드', '신고중량(KG)', '운송수단유형코드', '반입보세구역부호', 'HS10단위부호','관세율구분코드',
       '적출국가코드', '원산지국가코드']

for col in col_list:
    df_all[col] = df_all[col].astype('category')

In [ ]:
#1 stage autoencoder 학습, 테스트 데이터 생성

#1 stage에서는 우범(1)을 제외한 정상(0),핵심우범(2) 에대해서 분류만을 하기 때문에
#학습시 우범(1) 에대한 데이터는 삭제해주고 핵심우범 숫자 (2) -> (1) 로 변경해준다. 
df_all_auto = copy.deepcopy(df_all)

del df_all_auto['우범여부']

df_all_auto = df_all_auto[df_all_auto['핵심적발'] != 1]
df_all_auto.loc[(df_all_auto['핵심적발'] == 2),'핵심적발'] = 1

#mean 인코더가 오버피팅 가능성이 있어서 일부의 데이터로 인코더를 학습함

import category_encoders as ce

df_train, df_test = train_test_split(df_all_auto,test_size=0.3,random_state=13)

ce_model = ce.target_encoder.TargetEncoder(cols=col_list)
ce_model.fit(df_train[col_list],df_train.iloc[:,-1])

#mean 인코더 학습후 autoencorder 학습 때는 정상데이터가 많을수록 좋다 판단. 
#데이터 다시 스플릿 후 mean 인코딩
 
df_train, df_test = train_test_split(df_all_auto,test_size=0.1,random_state=13)

df_train[col_list] = ce_model.transform(df_train[col_list])

df_test[col_list] = ce_model.transform(df_test[col_list])



In [ ]:
#autoencorder 학습을위해 정상데이터만 분류
x_train = copy.deepcopy(df_train)
x_train = x_train[x_train['핵심적발'] == 0]
x_train = x_train.iloc[:,:-1]

x_test = copy.deepcopy(df_test)
x_test = x_test[x_test['핵심적발'] == 0]
x_test = x_test.iloc[:,:-1]

In [ ]:
#autoencorder 계층 생성

import tensorflow as tf
from tensorflow.keras.layers import Dense 
from tensorflow.keras import Model,Input,regularizers

leaky_relu = tf.nn.leaky_relu

input_dim = x_train.shape[1]
encoding_dim = 20

input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation=leaky_relu, )(input_layer)

latent_layer = Dense(25, activation=leaky_relu)(encoder)

decoder = Dense(encoding_dim, activation=leaky_relu)(latent_layer)

output = Dense(input_dim, activation=leaky_relu)(decoder)

autoencoder = Model(inputs=input_layer, outputs=output)

In [ ]:
#epoch, batch, optimizer, 손실함수, metric 설정
#학습시작
nb_epoch = 100
batch_size = 50
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])

history = autoencoder.fit(x_train, x_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, x_test),
                    verbose=1,)

In [ ]:
#2 stage lgbm 학습, 테스트 데이터 생성
#lgbm의 경우 따로 인코딩 없이 전체데이터로 학습 및 테스트
df_all_lgbm = copy.deepcopy(df_all)

del df_all_lgbm['핵심적발']

df_train_lgbm, df_test_lgbm = train_test_split(df_all_lgbm,test_size=0.2,random_state=13)

In [ ]:
#parameter 설정

parms = {
    'learning_rate' : 0.05,
    'n_estimators' : 500,
    'num_iterations' : 500,
    'max_depth' : -1,
    'n_jobs': -1,
    'scale_pos_weight' : 1.3,
    'early_stopping_rounds':30,
}

In [ ]:
#lgbm 모델 학습
lgbm_model_save = []

folds = KFold(n_splits=5, shuffle=True, random_state=1325)

for train_idx, valid_idx in folds.split(df_train_lgbm):

  lgbm = LGBMClassifier(**parms,random_state=1325)
  
  train_x, train_y = df_train_lgbm.iloc[train_idx,0:-1],df_train_lgbm.iloc[train_idx,-1]
  valid_x, valid_y = df_train_lgbm.iloc[valid_idx,0:-1],df_train_lgbm.iloc[valid_idx,-1]

  lgbm.fit(train_x,train_y,early_stopping_rounds=30,eval_set=[(valid_x,valid_y)],verbose=100) 

  lgbm_model_save.append(lgbm)

  print('==============')

  y_val = df_test_lgbm.iloc[:,-1]
  pred_val = lgbm.predict(df_test_lgbm.iloc[:,:-1])

  print('accuracy', metrics.accuracy_score(y_val,pred_val) )
  print('precision', metrics.precision_score(y_val,pred_val) )
  print('recall', metrics.recall_score(y_val,pred_val) )
  print('f1', metrics.f1_score(y_val,pred_val) )


In [ ]:
#soft voting 구현

temp = np.zeros((df_test_lgbm.shape[0],2))

for m in lgbm_model_save:
  predict_proba =  m.predict_proba(df_test_lgbm.iloc[:,:-1])
  temp += predict_proba/5
  
result = []

for i in temp:
  if i[0] > i[1]:
    result.append(0)
  else:
    result.append(1)


In [ ]:
#submission 위한 코드
#df_all 변수에 데이터 넣어주고 사용해야함
#코드 실행전 #데이터 전처리# 적혀있는 코드 실행해줘야함 

#1 stage

first_result = ce_model.transform(df_all)

auto_result = autoencoder.predict(first_result)

x_value = first_result.values

temp = []

for i in range(0,len(first_result)):
  temp.append(metrics.mean_squared_error(x_value[i], auto_result[i]))``

first_result['이상치'] = temp
first_result['이상치'] = first_result['이상치']*100000000
first_result = first_result[first_result['이상치']>300]

del first_result['이상치']
first_result['우범'] = 1


In [ ]:
# 2-stage lgbm, soft voting 사용하여 temp 변수에 정상, 우범 예측 확률 저장

temp = np.zeros((df_all.shape[0],2))

for m in lgbm_model_save:
  predict_proba =  m.predict_proba(df_all)
  temp += predict_proba/5

### probability 조정 ###

# 1-stage 핵심 우범으로 분류 된 데이터 한정으로 LGBM 정상 확률 0.8 이상이면 정상으로 변경

result = []

for val in temp[first_result.index]:

  if val[0] > 0.8 :
    result.append(0)
  else :
    result.append(1)

first_result['우범'] = result

# 2-stage 우범 확률 0.4 이상 우범으로 판정

result_2 = []

for val in temp:

  if val[1] > 0.4 :
    result_2.append(1)
  else :
    result_2.append(0)

# 1-stage 2-stage 결과 합치기

df_all['우범'] = result_2

df_all['우범'][first_result[first_result['우범'] == 0].index] = 0
df_all['우범'][first_result[first_result['우범'] == 1].index] = 1
